# Shapash Report

> The Shapash Report feature allows data scientists to deliver to anyone who is interested in their project **a document that freezes different aspects of their work as a basis of an audit report**. This document can be easily shared across teams and does not require anything else than a working internet connexion.

The shapash `generate_report` method allows to generate a report of your project.  
The result is a standalone HTML file that does not require any external dependency or server to work.  
The only requirement for the document to display properly is an active internet connexion.  

The report contains the following information :
1. General information about the project
2. Description of the dataset used
3. Documentation about data preparation and feature engineering
4. Details about your model used (library, parameters...)
5. Exploration of the data with a focus on the difference between train and test sets
6. Global explainability of the model
7. Model performance

> The first three points are generated using a YML file that the user should fill. An example is available [here](https://github.com/MAIF/shapash/blob/master/tutorial/report/utils/project_info.yml).

This tutorial presents an example of how one can generate the Shapash Report. 

Content:
- Set up an example project
- Create and fill your project information that will be displayed in the report
- Generate the base Shapash Report
- *Go further*: Generate a custom report

Data from Kaggle [House Prices](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

> Note : you may need to download the HTML report locally and open it in your browser otherwise it may not show properly.

In [8]:
import pandas as pd
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

## Building Supervized Model 

In [9]:
from shapash.data.data_loader import data_loading
house_df, house_dict = data_loading('house_prices')
y_df=house_df['SalePrice']
X_df=house_df[house_df.columns.difference(['SalePrice'])]

In [10]:
from category_encoders import OrdinalEncoder

categorical_features = [col for col in X_df.columns if X_df[col].dtype == 'object']

encoder = OrdinalEncoder(
    cols=categorical_features,
    handle_unknown='ignore',
    return_df=True).fit(X_df)

X_df = encoder.transform(X_df)

In [11]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X_df, y_df, train_size=0.75, random_state=1)

In [12]:
regressor = RandomForestRegressor(n_estimators=50).fit(Xtrain, ytrain)

In [13]:
y_pred = pd.DataFrame(regressor.predict(Xtest),columns=['pred'], index=Xtest.index)

## Fill your project information

**The next step is to create a YML file containing information about your project.**  

We will use the example file available [here](https://github.com/MAIF/shapash/blob/master/tutorial/report/utils/project_info.yml).  
**You are welcome to use this file as a template for your own report.**  

We display the information contained in the YML file below :

In [14]:
import yaml

with open(r'utils/project_info.yml') as file:
    project_info = yaml.full_load(file)

print(yaml.dump(project_info, sort_keys=False))

General information:
  version: 0.7
  name: House Prices Prediction Project
  purpose: Predicting the sale price of houses
  date: auto
  contributors: Yann Golhen, Sebastien Bidault, Thomas Bouche, Guillaume Vignal, Thibaud
    Real
  description: This work is a data science project that tries to predict the sale
    of houses based on 79 explanatory variables. It was designed inside the data science
    team at X. and improved since the beggining of the project in 2019. The model
    was put into production since February 2021.
  source code: https://github.com/MAIF/shapash/tree/master/tutorial
  Git commit: 1ff46e83beafba8949a7f3b7de27586acd6ae99e
Dataset information:
  path: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/
  origin: "The Assessor\xE2\u20AC\u2122s Office"
  description: the sale of individual residential property in Ames, Iowa
  depth: from 2006 to 2010
  perimeter: only residential sales
  target variable: SalePrice
  target description: The pr

---
**If you want to create your own custom file :**

The keys of the YML file are the titles of the different sections in the report.  
The YML file must then respect the following format:

```yaml
Title of section 1:  
    property1 name: property1 value 
    property2 name: property2 value 
    ...
Title of section 2:  
    property1 name: property1 value 
    ...
```
> Note that the **date** can be computed automatically using the *auto* property value (see example above)

## Generate your report

### Declare and compile SmartExplainer object

In [16]:
from shapash.explainer.smart_explainer import SmartExplainer

In [17]:
xpl = SmartExplainer(features_dict=house_dict) # optional parameter, specifies label for features name 

In [18]:
xpl.compile(
    x=Xtest,
    model=regressor,
    preprocessing=encoder, # Optional: compile step can use inverse_transform method
    y_pred=y_pred # Optional
)

Backend: Shap TreeExplainer


At this step the model can be checked and inspected using different methods of the SmartExplainer object we just created.  

Please refer to the other tutorials for more information.

### Generate the base Shapash Report

Next we can generate the report using the `generate_report` method of our SmartExplainer object.

We need to pass `x_train`, `y_train` and `y_test` parameters in order to explore the data used when training the model.

Please refer to the documentation for a full description of the parameters.


In [11]:
xpl.generate_report(
    output_file='output/report.html', 
    project_info_file='utils/project_info.yml',
    x_train=Xtrain,
    y_train=ytrain,
    y_test=ytest,
    title_story="House prices report",
    title_description="""This document is a data science report of the kaggle house prices tutorial project. 
        It was generated using the Shapash library.""",
    metrics=[
        {
            'path': 'sklearn.metrics.mean_absolute_error',
            'name': 'Mean absolute error', 
        },
        {
            'path': 'sklearn.metrics.mean_squared_error',
            'name': 'Mean squared error',
        }
    ]
)

INFO:papermill:Input Notebook:  C:\Users\subha\anaconda3\Lib\site-packages\shapash\report\base_report.ipynb
INFO:papermill:Output Notebook: C:\Users\subha\AppData\Local\Temp\tmp85_eund5\base_report.ipynb
INFO:blib2to3.pgen2.driver:Generating grammar tables from C:\Users\subha\anaconda3\lib\site-packages\blib2to3\Grammar.txt
INFO:blib2to3.pgen2.driver:Writing grammar tables to C:\Users\subha\AppData\Local\black\black\Cache\19.10b0\Grammar3.8.8.final.0.pickle
INFO:blib2to3.pgen2.driver:Writing failed: [Errno 2] No such file or directory: 'C:\\Users\\subha\\AppData\\Local\\black\\black\\Cache\\19.10b0\\tmpii5k0uwb'
INFO:blib2to3.pgen2.driver:Generating grammar tables from C:\Users\subha\anaconda3\lib\site-packages\blib2to3\PatternGrammar.txt
INFO:blib2to3.pgen2.driver:Writing grammar tables to C:\Users\subha\AppData\Local\black\black\Cache\19.10b0\PatternGrammar3.8.8.final.0.pickle
INFO:blib2to3.pgen2.driver:Writing failed: [Errno 2] No such file or directory: 'C:\\Users\\subha\\AppData\\

Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

INFO:papermill:Executing notebook with kernel: python3


> Note: You might want to specify the jupyter kernel used when generating the report.
You should consider using the `kernel_name` parameter to indicate what kernel to use.

## Customize your own report

Now let's customize our report by adding some new sections.

To do so :
- First, **copy the base report notebook** you can find [here](https://github.com/MAIF/shapash/blob/master/shapash/report/base_report.ipynb). This is the notebook that is used to generate the shapash report. It is executed and then converted to an HTML file. Only the output of each cell is kept and the code is deleted.
- Then, delete or add cells depending on what you want to change.
- Finally, add the parameter `notebook_path="path/to/your/custom/report.ipynb"` in the `generate_report` method.

> **Tip** : You can use the `working_dir` parameter to easily work inside your custom notebook before using the `generate_report` method. This way you can load the parameters used inside the notebook by papermill. Replace the `dir_path` inside your custom notebook with your own `working_dir` where are saved the different instances used.

For our simple example, we created [this notebook](https://github.com/MAIF/shapash/blob/master/tutorial/report/utils/custom_report.ipynb). 
- We removed the multivariate analysis using the `report.display_dataset_analysis(multivariate_analysis=False)` (see notebook utils/custom_report.ipynb for more information)
- It includes new sections **Relashionship with target variable** and **Relashionship between training variables** in which we included new simple graphs for this example. 
- We also added new cells at the end of the **metrics** section.

Next, we use this notebook to generate our new custom report :

In [19]:
xpl.generate_report(
    output_file='output/custom_reportv2.html', 
    project_info_file='utils/project_info.yml',
    x_train=Xtrain,
    y_train=ytrain,
    y_test=ytest,
    title_story="Sample report",
    title_description="""This document is a data science report of the kaggle house prices tutorial project. 
        It was generated using the Shapash library.""",
    metrics=[
        {
            'path': 'sklearn.metrics.mean_absolute_error',
            'name': 'Mean absolute error', 
        },
        {
            'path': 'sklearn.metrics.mean_squared_error',
            'name': 'Mean squared error',
        }
    ],
    working_dir='working',
    notebook_path="utils/custom_reportv2.ipynb"
)

INFO:papermill:Input Notebook:  utils/custom_reportv2.ipynb
INFO:papermill:Output Notebook: working\base_report.ipynb
INFO:blib2to3.pgen2.driver:Generating grammar tables from C:\Users\subha\anaconda3\lib\site-packages\blib2to3\Grammar.txt
INFO:blib2to3.pgen2.driver:Writing grammar tables to C:\Users\subha\AppData\Local\black\black\Cache\19.10b0\Grammar3.8.8.final.0.pickle
INFO:blib2to3.pgen2.driver:Writing failed: [Errno 2] No such file or directory: 'C:\\Users\\subha\\AppData\\Local\\black\\black\\Cache\\19.10b0\\tmpkyldiz_i'
INFO:blib2to3.pgen2.driver:Generating grammar tables from C:\Users\subha\anaconda3\lib\site-packages\blib2to3\PatternGrammar.txt
INFO:blib2to3.pgen2.driver:Writing grammar tables to C:\Users\subha\AppData\Local\black\black\Cache\19.10b0\PatternGrammar3.8.8.final.0.pickle
INFO:blib2to3.pgen2.driver:Writing failed: [Errno 2] No such file or directory: 'C:\\Users\\subha\\AppData\\Local\\black\\black\\Cache\\19.10b0\\tmpphytvjo0'


Executing:   0%|          | 0/28 [00:00<?, ?cell/s]

INFO:papermill:Executing notebook with kernel: python3
